In [1]:
import pm4py
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

print(matplotlib.__version__)

3.6.2


In [3]:
#@title add label to dataset

# variant = xes_importer.Variants.ITERPARSE
# parameters = {variant.value.Parameters.TIMESTAMP_SORT: True}
# log = xes_importer.apply('Helpdesk.xes',parameters=parameters)

# from pm4py.objects.conversion.log import converter as log_converter
# df = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)

# activities = {
#     'Insert ticket':0,'Assign seriousness':1, 'Take in charge ticket':2, 'Resolve ticket':3,'Closed':4, 'Wait':5, 'Create SW anomaly':6, 
#        'Schedule intervention':7, 'RESOLVED':8, 'INVALID':9, 'VERIFIED':10,'Resolve SW anomaly':11, 'Require upgrade':12, 'DUPLICATE':13 }

# # df['Activity'] = df['Activity'].apply(lambda x: activities.get(x, -1) if x else -1)
# # df['case:concept:name'] = df['case:concept:name'].str[4:]

# # df.drop(columns=['Unnamed: 0', 'concept:name', 'lifecycle:transition', 'org:resource', 'case:variant','Resource', 'case:creator','case:variant-index'] , inplace=True)
# df.drop(columns=['Unnamed: 0'] , inplace=True)
# df.rename(columns={'time:timestamp':'timestamp', 'Activity':'activity', 'case:concept:name':'case'}, inplace=True)
# df_ = df.copy()
# df['date'] = df['timestamp'].dt.date
# df['hour'] = df['timestamp'].dt.hour
# df['dayofweek'] = df['timestamp'].dt.dayofweek+1
# df['dayofmonth'] = df['timestamp'].dt.day
# df['dayofyear'] = df['timestamp'].dt.dayofyear
# df['weekofyear'] = df['timestamp'].dt.isocalendar().week



/Users/yousef/miniforge3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|████| 4580/4580 [00:00<00:00, 8693.84it/s]


In [3]:
import pandas as pd 

def make_dataset(dataset, iloc_from, iloc_to):
  df_ = dataset[['text','wip']].iloc[iloc_from:iloc_to]
  df_ .columns = ["text", "labels"]
  df_.reset_index(inplace=True,drop=True)
  return df_

# train_size = int(df.shape[0] / 5)*2
train_size = 10
eval_size = train_size
test_size = train_size
# test_size = df.shape[0]-train_size-eval_size

print(f'train, eval, test size: {train_size, eval_size, test_size}')


df = pd.read_csv('df_nlp_real.csv',usecols=['wip','text'],dtype={'text':'string','wip':'float'})
train_df = make_dataset(df, 0,train_size)
eval_df = make_dataset(df, train_size,train_size+eval_size)
test_df = make_dataset(df, train_size+eval_size, train_size+eval_size+eval_size)
display(train_df.head(3))
# display(eval_df)
# display(test_df)


train, eval, test size: (10, 10, 10)


,text,labels
0,R2 starts A1 on C3608 at 2010-01-13 06:40,0.0
1,R2 starts A1 on C2748 at 2010-01-13 10:26,0.0
2,R2 starts A1 on C4284 at 2010-01-13 10:30,0.0


In [1]:
import numpy as np
import pandas as pd
import transformers
from datasets import Dataset,load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
